In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [8]:
df = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_sample = pd.read_csv('../input/sample_submission.csv')

In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 12))
ax.scatter(df.pickup_longitude.values, df.pickup_latitude.values, s=5, color='red', alpha=0.5)
ax.scatter(df.dropoff_longitude.values, df.dropoff_latitude.values, s=5, color='black', alpha=0.5)
ax.set_xlim([-74.05, -74.00])
ax.set_ylim([40.70, 40.75])

In [ ]:
df = df.loc[df['pickup_longitude']> -74.02]
df = df.loc[df['dropoff_longitude']> -74.02]

In [ ]:
df = df[(df['trip_duration'] > 60) & (df['trip_duration'] < 3600 * 24)]

df['hour'] = df['pickup_datetime'].apply(lambda x: int(x.split()[1][0:2]))

df['dist'] = np.sqrt((df['pickup_latitude']-df['dropoff_latitude'])**2 + (df['pickup_longitude']-df['dropoff_longitude'])**2) 

In [ ]:
df = df.loc[df['trip_duration']< 1500000]

In [ ]:
# choix de la target et des features

y = df["trip_duration"] # <-- target
X = df[["passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude","dist","hour"]] # <-- features

In [ ]:
# importer la lib pour la regression de Random Forest

from sklearn.ensemble import RandomForestRegressor

In [ ]:
# declarer le model et l'entrainer

rf = RandomForestRegressor(n_estimators=20)
rf.fit(X, y)

In [ ]:
# importer la lib pour cross valider le model

from sklearn.model_selection import cross_val_score

In [ ]:
# calculer les scores de cross validation du model selon une decoupe en 5 du dataset de train

cv_scores = cross_val_score(rf,X,y,cv=5,scoring='neg_mean_squared_log_error')

In [ ]:
for i in range(len(cv_scores)):
    cv_scores[i] = np.sqrt(abs(cv_scores[i]))
print(np.mean(cv_scores))

In [ ]:
df_test['hour'] = df_test['pickup_datetime'].apply(lambda x: int(x.split()[1][0:2]))

df_test['dist'] = np.sqrt((df_test['pickup_latitude']-df_test['dropoff_latitude'])**2 + (df_test['pickup_longitude']-df_test['dropoff_longitude'])**2) 
df_test.head()

In [ ]:
X_test = df_test[["passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude","dist","hour"]]
predicted_duration = rf.predict(X_test)
print(predicted_duration)

In [ ]:
df_sample.head(3)

In [ ]:
my_submission = pd.DataFrame({'id': df_test.id, 'trip_duration': predicted_duration})
print(my_submission)

In [ ]:
my_submission.to_csv('submission.csv', index=False)